# Homework #3

# $ \\ $

# Problem 1: Obtain structured company data using Regex

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import re
import pandas as pd

In [5]:
wiki_df = pd.read_csv('data/kdwd_r1k_articles.csv')
wiki_df.head(2)

,page_id,page_title,page_views,intro_text
0,856,Apple Inc.,190485,Apple Inc. is an American multinational techno...
1,2386,American Airlines,40829,"American Airlines, Inc. (AA) is a major Americ..."


### -) Write a regex to find unusually capitalized terms
Sometimes product names will have unusual capitalization such as iPhone or ThinkPad. Find a list of such terms and investigate if you think some of them are products.

In [11]:
maybe_products_ptn = r'\b[a-zA-Z]*[a-z][A-Z][a-zA-Z]*\b'  # updated pattern to include lowercase start

# code here!
maybe_products_set = set()
for text in wiki_df['intro_text']:
    maybe_products_set.update(re.findall(maybe_products_ptn, text))

print(len(maybe_products_set), 'terms found that are potential products')
print(maybe_products_set)

256 terms found that are potential products
{'UnionPay', 'CubeSmart', 'BioSpin', 'iLife', 'PowerTOP', 'PulteGroup', 'ProPublica', 'RingCentral', 'ThinkPad', 'CareFusion', 'McCann', 'HomeTeam', 'McKelvey', 'RenaissanceRe', 'DDoS', 'SaskTel', 'OkCupid', 'McVey', 'LogMeIn', 'NetScreen', 'SoHo', 'CityCenter', 'McGee', 'AirPods', 'SystemC', 'RiverSource', 'SoCs', 'SoftBank', 'SuperTarget', 'AbbVie', 'FreeStyle', 'PepsiCo', 'LinkedIn', 'AbioCor', 'MinuteClinic', 'McMaster', 'BioWare', 'BofA', 'McMoRan', 'FarmVille', 'FlexRigs', 'ServiceNow', 'jetBlue', 'SpectraSite', 'BankAmerica', 'WellCare', 'McCormick', 'GrafTech', 'tvOS', 'MitraClip', 'ExpertCity', 'LoRa', 'CoreSite', 'TransFund', 'iTunes', 'MidCap', 'ViacomCBS', 'MullenLowe', 'SoCalGas', 'OpenStack', 'FactSet', 'SeaTac', 'BellSouth', 'GeForce', 'kV', 'AmeriGas', 'VoIP', 'DoubleTree', 'ManpowerGroup', 'HoloLens', 'AmerisourceBergen', 'KGaA', 'SolarCity', 'HomeAway', 'IoT', 'FirstEnergy', 'AspenTech', 'FoodSaver', 'MiWi', 'eSignature', 'D

In [10]:
# verify that we found some of the key terms
assert('iPhone' in maybe_products_set)
assert('ThinkPad' in maybe_products_set)

### -) Parse company acquisition data from plain text

We are looking to identify the following types of patterns.<br />
`'Citrix acquired Sequoia Software Corp'`<br />
`'Moody\'s was acquired by Dun & Bradstreet in 1962.'`<br />
The idea here is to look for patterns around the word 'acquire' with two valid entities on either side, and an option year at the end.<br />
<span style="color:orange">Helpful Reminder:</span> you can create non-capturing capture groups via `(?:capture this|or that)`

In [36]:
# find a way to capture, 'FedEx', 'Coca-Cola', 'Sequoia Software Corp', 'Dun & Bradstreet' and 'Moody\'s'
company_ptn = r'\b(?:(?:[A-Z][a-z\']*)+|[A-Z]+)(?:(?:[ -](?:&|and)?[ -]?(?:(?:[A-Z][a-z\']*)+|[A-Z]+))*)(?: (?:Corporation|Corp|Group|Inc|LLC|Ltd|Software|Systems))?\b'

# code here!
maybe_companies_set = set()
for text in wiki_df['intro_text']:
    maybe_companies_set.update(re.findall(company_ptn, text))

print(len(maybe_companies_set), 'terms found that are potential companies')
print(maybe_companies_set)

6359 terms found that are potential companies
{'East West Bancorp Inc', 'Black & Decker', 'Loom', 'Mobil', 'Sara Lee', 'Founding', 'Financial Services', 'Pork', 'Fast Tech', 'Rayovac and Varta', 'On October', 'Green Building Council', 'Conshohocken', 'Wireless BLE Bluetooth Low-Energy and USB', 'Darden Restaurants Inc', 'ICE Clear Netherlands and ICE NGX', 'Robert Half', "Nature's Miracle", 'Southern California', 'Cotai', 'EA Vancouver', 'NOV', 'AMD', 'Hess', 'Phase I-IV', 'Old El Paso', 'Aerospace', 'Restasis', 'Permian Basin', 'Canada and Europe', 'Chevrolet', 'CEO Gil Amelio', 'Holden', 'Royal Vopak', 'Jeff Fettig', 'Scotland Group', 'HII', 'Private First Class Stuart S', 'Jack Dorsey', 'Intercontinental Exchange', 'BNP Paribas', 'Indiana and Virginia', "Tate's Bake Shop and Tang", 'Defense', 'Frost Bankers', 'Healthpeak Properties', 'LDS Church', 'Exploration and Production', 'Madison Square', 'NCR Global Headquarters', 'Broadcom Inc', 'MTN', 'Conversely', 'Quanta Services', 'NJ', 

In [41]:
# verify that we found some of the key terms
assert('FedEx' in maybe_companies_set)
assert('Coca-Cola' in maybe_companies_set)
assert('Sequoia Software Corp' in maybe_companies_set)
assert('Dun & Bradstreet' in maybe_companies_set)
assert('Moody\'s' in maybe_companies_set)

In [42]:
# acquisition pattern
acquisition_ptn = r'(?:acquired|was acquired by)'

# find a way to optionally capture the year such as ' in 1962'
optional_year_ptn = r'(?: in [12][0-9]{3}\b)?'

In [43]:
# let's assemble the patterns together to a full capture pattern
full_acquisition_pattern = (
    company_ptn + r'\s+' + acquisition_ptn + r'\s+' + company_ptn + optional_year_ptn
)

In [44]:
acquisition_strings = []
for _, row in wiki_df.iterrows():
    acquisition_strings.extend(re.findall(full_acquisition_pattern, row['intro_text']))
print(len(acquisition_strings), 'potential acquisitions found.')

22 potential acquisitions found.


In [45]:
# verify that we found some of the key terms
assert('Citrix acquired Sequoia Software Corp' in acquisition_strings)
assert('Moody\'s was acquired by Dun & Bradstreet in 1962' in acquisition_strings)

In [46]:
print(acquisition_strings)

['Citrix acquired Sequoia Software Corp', 'Citrix acquired Framehawk', 'Aflac acquired Continental American Insurance Company', 'Giannini acquired Banca', 'It was acquired by Avago Technologies in 2016', "Moody's was acquired by Dun & Bradstreet in 1962", 'Brookfield acquired Rouse Properties in 2016', 'Medtronic acquired Irish', 'ACE Limited acquired Chubb in 2016', 'Lottomatica acquired Gtech Corporation', 'CIT Group acquired OneWest Bank', 'Advance Auto Parts acquired Carport Auto Parts', 'Advance acquired Discount Auto Parts', 'Advance Auto Parts acquired BWP Distributors', 'Thermo Fisher acquired Life Technologies Corporation', 'Bell Atlantic acquired GTE', 'It was acquired by FIS', 'Keurig Green Mountain acquired Dr Pepper Snapple Group', 'Gendex Corporation acquired Dentsply International Inc', 'Sovran Self Storage acquired LifeStorage LP', "Novartis acquired Incyte's", 'Colfax acquired British Charter International PLC']


### -) Question: Are there any false positives in your results? If so, how could you account for them?

**ANSWER**

The following are false positives:
- It was acquired by Avago Technologies in 2016
- It was acquired by FIS
  
It is not a company name. We could account for these false positives by limiting the company names to be at least 3 characters long.

### -) Let's look into speed of regex matching

In [47]:
# write a simple pattern of your choice to search for in our dataset
search_ptn = r'iPhone'

In [48]:
# assemble a list of strings
doc_list = wiki_df['intro_text'].tolist()

In [50]:
%%timeit
# evaluation loop here
for doc in doc_list:
    re.search(search_ptn, doc)

570 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Let's compile the regex and see if this increases the speed using `re.compile`

In [51]:
%%timeit
# evaluation loop here
compiled_search_ptn = re.compile(search_ptn)
for doc in doc_list:
    compiled_search_ptn.search(doc)

399 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### A note on speed: basic string operations are always faster than regex
#### show this using `'my_string' in 'other_string'`

In [54]:
%%timeit
# code here!
for doc in doc_list:
    if 'iPhone' in doc:
        pass

266 µs ± 2.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Problem 2: Word phrases

### In this problem we will look at methods to identify valid n-grams such as 'New York' or 'Barack Obama' while eliminating statistical flukes such as `in the` or `i write`.

### Preprocessing such as this can drastically improved embeddings since words can ngrams will often have a different meaning than the sum of its parts
### `V('united')` + `V('states')` != `V('united states')`
### `V('real')` + `V('estate')` != `V('real estate')`

In [55]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/bluebird/.pyenv/versions/3.10.11/envs/venv310/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [56]:
import os
import json
import re

import numpy as np
import pandas as pd

In [58]:
wiki_df = pd.read_csv('data/kdwd_r1k_articles.csv')

### get consecutive unigrams for the 'intro_text' column of our dataset

In [61]:
# list of lists of unigrams
unigram_pattern = r'\b\w+\b'
corpus = [re.findall(unigram_pattern, doc.lower()) for doc in wiki_df['intro_text'].tolist()]
print(corpus)

[['apple', 'inc', 'is', 'an', 'american', 'multinational', 'technology', 'company', 'headquartered', 'in', 'cupertino', 'california', 'that', 'designs', 'develops', 'and', 'sells', 'consumer', 'electronics', 'computer', 'software', 'and', 'online', 'services', 'it', 'is', 'considered', 'one', 'of', 'the', 'big', 'four', 'tech', 'companies', 'along', 'with', 'amazon', 'google', 'and', 'facebook', 'the', 'company', 's', 'hardware', 'products', 'include', 'the', 'iphone', 'smartphone', 'the', 'ipad', 'tablet', 'computer', 'the', 'mac', 'personal', 'computer', 'the', 'ipod', 'portable', 'media', 'player', 'the', 'apple', 'watch', 'smartwatch', 'the', 'apple', 'tv', 'digital', 'media', 'player', 'the', 'airpods', 'wireless', 'earbuds', 'and', 'the', 'homepod', 'smart', 'speaker', 'apple', 's', 'software', 'includes', 'the', 'macos', 'ios', 'ipados', 'watchos', 'and', 'tvos', 'operating', 'systems', 'the', 'itunes', 'media', 'player', 'the', 'safari', 'web', 'browser', 'the', 'shazam', 'acou

### The package `gensim` has a convenient wrapper to obtain statistically significant ngrams/Phrase automatically

### we need to first `pip install gensim`
### `gensim` is a useful library for anything related to word representations and embeddings. It will come up a few more times. https://radimrehurek.com/gensim/index.html

In [62]:
from gensim.models.phrases import Phrases

### Write some code to parse our corpus and use valid ngrams using `Phrases`

In [63]:
phrases = Phrases(corpus, min_count=5, threshold=10)
print(phrases)

Phrases<60670 vocab, min_count=5, threshold=10, max_vocab_size=40000000>


In [ ]:
vocab_count_dict = {k: v for k, v in phrases.vocab.items()}

n_grams = ...

In [ ]:
print(n_grams.shape[0], 'n-grams found')
n_grams.head(10)

### How do the results look? Can you improve the results by excluding common terms using the `connector_words` kwarg of `Phrases`?

In [ ]:
phrases = ...

In [ ]:
n_grams = ...

In [ ]:
print(n_grams.shape[0], 'n-grams found')
n_grams.head(10)

## This was convenient, but it's also a black box where many of the knobs for tuning are actually broken in the newest version. Let's try to create our own solution for finding n-grams.

### To do this, let's start by counting unigrams and bigrams within our corpus

### Tip: use Counter for easy counting. It behaves similar to a dictionary with some added functionality around counting. such as `my_counter[unknown_key]` returning `0` for all unknown keys

In [ ]:
from collections import Counter

unigram_counter = Counter()
bigram_counter = Counter()
for tokens in corpus:
    # your code here

### Now we need to come up with a score for each bigram that helps us decide on its importance and the fact of whether it is truly a bigram or two independent unigrams.

In [ ]:
# your code here

bigram_df = ...

In [ ]:
bigram_df.head(10)

### Find ways to sort and filter your output to bigrams that make sense, such as `wells fargo`, `apple inc` or `puerto rico`

In [ ]:
# your code here

filtered_bigram_df = ...

In [ ]:
filtered_bigram_df.head(10)

# Problem 3: Corporate Similarity and Returns
### In this example we'll explore how to use NLP to measure corporate similarity

### In particular we will
 - ### Make word vectors for firms in order to get an NLP measure of similarity
 - ### Measure the quality of this similarity metric by predicting future co-movement of returns. 
 
## Step X: This problem uses a few concepts of basic modeling such as `sklearn.model_selection.train_test_split` and `sklearn.linear_model.LinearRegression`
 

# $ \\ $
## Step 0: Load the MD&A section from Form-10-K from 2016

In [ ]:
with open('../../data/parsed_mda.json') as f:
    data = json.load(f)

### Next, take only the first filing for each company

In [ ]:
clean = []
seen = set()
for item in data:
    if item['ticker'] in seen:
        continue
    else:
        seen.add(item['ticker'])
        clean.append(item)
data = clean
del clean

### Now load the price data for 2015-2018

In [ ]:
prices = pd.read_csv(
    '../../data/sp500_prices.csv', 
    index_col=0, 
    parse_dates=True
).loc['2015-01-01':'2018-01-01']

In [ ]:
prices.head()

In [ ]:
data_tickers = [item['ticker'] for item in data]
assert len(data_tickers) == len(set(data_tickers)), 'non-unique tickers, this will not work'

## Step 1: clean the text
### Much of NLP boils down to doing reasonable processing on text.
### First, we'll try out very minimial processing

In [ ]:
def clean_mda_simple(mda):
    return mda.lower()

In [ ]:
# add import here

In [ ]:
word_vecs = ...

## Step 2: Pairwise Word similarity
### Calculate the pariwise cosine similarity between word vectors
### Make the cosine similarities into a dataframe indexed/columned on ticker symbols

In [ ]:
word_sims = ...

## Step 2a: Why `cosine_similarity` and not another measure?

In [ ]:
# ANSWER GOES HERE

## Step 3: Wrangle the price and word data
### Our goal here is to have a dataframe which is indexed on PAIRS of tickers and has columns
 - ### `returns_correlation`: the correlation of returns for those two tickers from Jan 1 2016 to Jan 1 2017
 - ### `word_similarity`: the cosine similarity of the word vectors for the two companies' MD&A sections
 
## Tips
 - ### NB: use pct_change to calculate returns in pandas
 - ### NB: use the pandas builtin corr function to calculate correlations (we don't need anything fancy)
 - ### NB: the index of the dataframe should have two columns (the tickers)

In [ ]:
# one way you might do this is
rets_cor = ... # calculate returns correlations
word_cor = #  calcuate the word similarities in the right shape

all_data = rets_cor.join(word_cor)
all_data = all_data.dropna()
all_data.head()

## Step 3a: 
 - ### What is the contemperaneous correlation of these data?
 - ### Make a scatter plot of the returns correlation and word similarities

In [ ]:
all_data.corr()

In [ ]:
## This should be about 12%. That's not bad, but we can do better

In [ ]:
all_data.plot.scatter(x='returns_correlation', y='word_similarity')

## Part 4: Try to predict the future returns correlations
### Use OLS (`LinearRegression`) to predict `returns_correlation` from `word_similarity`. 
### What is the (contemperaneous) out of sample performance?

In [ ]:
data_df =  ...
feature_cols =  ...
target_col =  ...

In [ ]:
# add code here

reg = ...# add code here
reg.score(X_test, y_test)

In [ ]:
pd.Series(reg.coef_, index=feature_cols)

### This is not amazing. We can do better!

## $ \\ $ 

## Part 5: Repeat, but be careful
### Here we will see if we can clean the data better

### Things to try
 - ### Look at the histograms of word similarities to see if we can "ignore" some ill-behaved data
 - ### Try limiting how greedy the `TFIDFVectorizer` is: `min_df`, `max_df`, `max_features`, etc.
 
### We will examine our data and look for things that look out of place
 - ### We will ultimately want our data to look normally distributed


In [ ]:
def clean_mda(mda):
    paras = [p.lower() for p in mda.split('\n') if len(p) > 40]
    cleaned =  ' '.join(paras)
    words = cleaned.split()
    words = [word for word in words if len(word) > 2]
    if len(words) > 10:
        return ' '.join(words)
    else:
        return ''

In [ ]:
vec = TfidfVectorizer(
    min_df=...
    max_df= ...
    max_features= ...
)
word_vecs = vec.fit_transform((clean_mda(item['mda']) for item in data))


word_sims =  ...
# Lots of word similarities are all zeros- so we'll ignore
# add code here to remove rows of word_sims where all the elements are zero

In [ ]:
# calculate the returns correlation and the cosine similarities as above
all_data = ...

In [ ]:
# inspect your data- make some histograms
plt.figure(figsize=(12,7))
all_data.returns_correlation.hist(bins=40)
plt.title('Returns Correlation')
plt.figure(figsize=(12,7))
all_data.word_similarity.hist(bins=40)
plt.title('Word Similarity')

### Cleaning our data
### It seems lots of things are identically 0 (no word overlap) or identically 1 (the MD&A section for one company perfectly overlaps itself). We will exclude those

In [ ]:
# your code here

In [ ]:
# examine histograms again
plt.figure(figsize=(12,7))
all_data.returns_correlation.hist(bins=40)
plt.title('Returns Correlation')
plt.figure(figsize=(12,7))
all_data.word_similarity.hist(bins=40)
plt.title('Word Similarity')

In [ ]:
all_data.corr()

### Lastly, there is a bit of a "hump" at low `word_similarity`

In [ ]:
# add code here
all_data.corr()

### The contemperaneous correlation is twice as large!

## Part 6: Now, repeat the exercise of predicting future returns correlation

In [ ]:
data_df =  ...# Add code here
# add code here

In [ ]:
reg =  ...
reg.score(X_test, y_test)

In [ ]:
pd.Series(reg.coef_, index=feature_cols)

### This is about 5 times better than before!
## $ \\ $ 
## Part 7: What will happen if we include last year's returns correlation as a feature

In [ ]:
last_year_corr =  ...
data_df = last_year_corr.join( ...

In [ ]:
reg =  ...
reg.score(X_test, y_test)

In [ ]:
pd.Series(reg.coef_, index=feature_cols)

## Indeed, we do much better, but the word features still help!